In [ ]:
import urllib.request
import os
import cv2 as cv
from IPython.display import Image, display
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import mediapipe as mp

In [ ]:
# modi

In [ ]:
# mm

In [ ]:

# Load pre-trained face cascade
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
#face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_profileface.xml')

def process_image_casc(image_path):
    img = cv.imread(image_path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05,  # Smaller = more detections but slower (try 1.05-1.4)
    minNeighbors=6,   # Lower = more detections but more false positives (try 3-6)
    minSize=(1, 1)  # Minimum face size to detect
                                         )
    
    has_face = len(faces) > 0
    
    if has_face:
        for (x, y, w, h) in faces:
            cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        plt.imshow(img)
        plt.show()
        
    return has_face, img

In [ ]:
os.getcwd()

In [ ]:
root_path = Path(os.getcwd())

In [ ]:
root_path

In [ ]:

# Define paths
image_dir = root_path/"../test_data"  # Replace with your directory containing images
output_dir_with_face = root_path/"../test_with_face"  # Replace with output directory for images with persons
output_dir_without_face = root_path/"../test_without_face"  # Replace with output directory for images without persons


In [ ]:
# Create output directories
#os.chdir(root_path/'..')
os.makedirs(output_dir_with_face, exist_ok=True)
os.makedirs(output_dir_without_face, exist_ok=True)
#os.chdir('root_path')

In [ ]:
images = os.listdir(image_dir)
len(images)

In [ ]:
if '.DS_Store' in images:
  images.remove('.DS_Store')


In [ ]:
images[0]

In [ ]:
image_path = image_dir/images[0]
image_path

In [ ]:
has_face, img = process_image_casc(image_path)
plt.imshow(img)

In [ ]:
results = []
images_processed = []
round_counter = 0

images = Path(image_dir).glob("*.tif")
for image in images:
    print(round_counter)
    image_path = image_dir/image
    has_face, img = process_image_casc(image_path)
    results.append(has_face)
    images_processed.append(img)
    round_counter += 1

In [ ]:
results_imgs = pd.DataFrame({'images_processed': images_processed, 'results': results})
results_imgs.head()

In [ ]:
plt.imshow(results_imgs[results_imgs.results].iloc[1,0])

In [ ]:
sum(results)

In [ ]:
# Define paths
image_dir = root_path/"../test_data"  # Replace with your directory containing images
output_dir_with_face = root_path/"../test_with_face"  # Replace with output directory for images with persons
output_dir_without_face = root_path/"../test_without_face"  # Replace with output directory for images without persons


In [ ]:
from mtcnn import MTCNN
mtcnn = MTCNN()

def detect_faces_mtcnn(image):
    faces = mtcnn.detect_faces(image,
                              min_face_size=10,  # Detect smaller faces
    threshold_pnet=0.6,  # More proposals from PNet
    threshold_rnet=0.7,  # Loosen RNet filtering
    threshold_onet=0.8   # More final faces accepted by ONet
                              )
    return len(faces) > 0, faces

In [ ]:
def process_faces(image_dir, output_dir_with_face, output_dir_without_face):
   images = Path(image_dir).glob("*.tif")
   for img_path in images:
       # Read and process image
       image = cv.imread(str(img_path))
       has_faces, faces = detect_faces_mtcnn(image)
       
       if has_faces:
           # Draw boxes
           for face in faces:
               x, y, w, h = face['box']
               cv.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
           
           # Display
           plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
           plt.show()
           
           # Save
           img_path_output = str(img_path).replace('test_data', 'test_with_face')
           new_path = str(img_path_output).replace('.tif', '_mtcnn.tif')
           print(new_path)
           cv.imwrite(new_path, image)
       else:
           img_path_output = str(img_path).replace('test_data', 'test_without_face')
           new_path = str(img_path_output).replace('.tif', '_mtcnn.tif')
           print(new_path)
           cv.imwrite(new_path, image)

In [ ]:
output_dir_with_face

In [ ]:
output_dir_without_face

In [ ]:
process_faces(image_dir, output_dir_with_face, output_dir_without_face)

In [ ]:
images = Path(image_dir).glob("*.tif")
for image in images:
    image = cv.imread(image)
    length, output = detect_faces_mtcnn(image)
    print(length)
    print(output)
    plt.imshow(image)
    plt.show()

In [ ]:
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(
    model_selection=0.8,  # 0 for close-range, 1 for far-range detection
    min_detection_confidence=0.1  # Lower this to detect more faces (0.1-0.9)
)

def detect_faces(image):
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)
    if results.detections:
        print(f"Found {len(results.detections)} face(s)")
        return results.detections
    else:
        print("No faces detected")
        return None

In [ ]:
round_counter = 0
images = Path(image_dir).glob("*.tif")
for image in images:
    if round_counter > 1:
        break
    image = cv.imread(image)
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    detect_faces(image_rgb)
    round_counter += 1